## 搭建单层多个神经元进行训练（感知机）

In [1]:
import pickle
import os
import numpy as np
CIFAR_DIR = "../data/cifar-10-batches-py"

In [2]:
for file in os.listdir(CIFAR_DIR):
    print(file.title())
print(os.path.join(CIFAR_DIR,os.listdir(CIFAR_DIR)[1].title()))

Batches.Meta
Data_Batch_1
Data_Batch_2
Data_Batch_3
Data_Batch_4
Data_Batch_5
Readme.Html
Test_Batch
../data/cifar-10-batches-py\Data_Batch_1


In [3]:
def load_single_data(filename):
    """读取单个数据集中的数据"""
    with open(filename, 'rb') as one_file:
        batch_data = pickle.load(one_file, encoding = "bytes")
        return batch_data[b'data'], batch_data[b'labels']
    
# 数据读取处理
class CIFAR10_Read:
    def __init__(self, filenames, need_shuffle):
        """从文件列表中读取数据"""
        all_data = []
        all_labels = []
        for filename in filenames:
            """将读取的数据存入数组中"""
            data, labels = load_single_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = len(self._data)
        self._need_shuffle  = need_shuffle
        self._index = 0
        if need_shuffle:
            self.shuffle_data()
            
    def shuffle_data(self):
        """将数据进行乱序处理"""
        p_index = np.random.permutation(self._num_examples)
        self._data = self._data[p_index]
        self._labels = self._labels[p_index]
    
    def next_batch(self, batch_size):
        """读取下一组数据集"""
        if batch_size > self._num_examples:
            raise Exception("batch size is larger than examples' num")
        end_index = self._index + batch_size
        if end_index > self._num_examples:
            if self._need_shuffle:
                """如果当前数据读取完毕，且如果可以对数据进行重排列，则重新组合数据集"""
                self.shuffle_data()
                self._index = 0
                end_index = batch_size
            else:
                raise Exception("there is no more data")
        batch_data, batch_labels = self._data[self._index:end_index], self._labels[self._index:end_index] 
        self._index = end_index
        return batch_data, batch_labels
        

In [4]:
from tensorflow import compat as tf_compat
tf = tf_compat.v1
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## 搭建多个神经元的计算图

In [5]:
# 输入数据
x = tf.placeholder(tf.float32, [None, 3072])
# 输出标签
y = tf.placeholder(tf.int64, [None])
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                    initializer = tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [10], initializer = tf.constant_initializer(0.0))
# 计算输出值
y_ = tf.matmul(x, w) + b
# 进行概率映射
p_y = tf.nn.softmax(y_)
# 使用one_hot编码
y_one_hot = tf.one_hot(y, 10, dtype = tf.float32)
# 计算损失函数
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
# 从计算的结果中，获取最大值的索引作为预测的结果值
predict = tf.argmax(y_, 1)
# 计算平均正确率
correct_predict = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))
    
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
initializer =  tf.global_variables_initializer()
batch_steps = 100000
batch_size = 50
test_steps = int(2000 / batch_size)
train_filenames = [os.path.join(CIFAR_DIR, "Data_Batch_%d" % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, "Test_Batch")]
print(train_filenames)
train_data_reader = CIFAR10_Read(train_filenames, True)
def print_log(step, current_loss_val, acc, is_test = False):
    if is_test:
        mode = 'Test'
    else:
        mode = 'Train'
    print("[%s] Step %d, loss: %4.5f, acc: %4.5f." %(mode, step, current_loss_val, acc))
    
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# 初始化所有的变量数据
    sess.run(initializer)
    for one_train_step in range(batch_steps):
        one_batch_data, one_batch_labels = train_data_reader.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                        feed_dict={
                                            x: one_batch_data,
                                            y: one_batch_labels
                                        })
        if (one_train_step + 1) % 500 == 0:
            print_log(one_train_step+1, loss_val, acc_val)
        if (one_train_step + 1) % 2000 == 0:
            test_data = CIFAR10_Read(test_filenames, False)
            one_test_mean_acc=[]
            one_test_mean_loss =[]
            for one_test_step in range(test_steps):
                one_test_batch_data, one_test_batch_labels = test_data.next_batch(batch_size)
                test_loss_val, test_acc_val = sess.run(
                    [loss, accuracy],
                    feed_dict={
                        x: one_test_batch_data,
                        y: one_test_batch_labels
                    }
                )
                one_test_mean_acc.append(test_acc_val)
                one_test_mean_loss.append(test_loss_val)
            print_log(one_train_step + 1, np.mean(one_test_mean_loss),
                      np.mean(one_test_mean_loss), True)
            
    
        
    
    

['../data/cifar-10-batches-py\\Data_Batch_1', '../data/cifar-10-batches-py\\Data_Batch_2', '../data/cifar-10-batches-py\\Data_Batch_3', '../data/cifar-10-batches-py\\Data_Batch_4', '../data/cifar-10-batches-py\\Data_Batch_5']
(50000, 3072) (50000,)
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

[Train] Step 500, loss: 0.13304, acc: 0.30000.
[Train] Step 1000, loss: 0.12940, acc: 0.34000.
[Train] Step 1500, loss: 0.15728, acc: 0.18000.
[Train] Step 2000, loss: 0.12963, acc: 0.32000.
(10000, 3072) (10000,)
[Test] Step 2000, loss: 0.14242, acc: 0.14242.
[Train] Step 2500, loss: 0.11708, acc: 0.40000.
[Train] Step 3000, loss: 0.15661, acc: 0.20000.
[Train] Step 3500, loss: 0.14737, acc: 0.24000.
[Train] Step 4000, loss: 0.15394, acc: 0.22000.
(10000, 3072) (10000,)
[Test] Step 4000, loss: 0.12965, acc: 0.12965.
[Train] Step 4500, loss: 0.11268, acc: 0.42000.
[Train] Step 5000, loss: 0.